In [1]:
import sys
sys.path.insert(0, 'C:\\Users\\Paarth Tandon\\Desktop\\repos\\personal-music-gen')

In [2]:
import torch
import torchaudio
from audiocraft.models import MusicGen
from personal_musicgen.data.datasets import AudioDataset

data = AudioDataset('../data/chunks_original/', no_label=False)
model = MusicGen.get_pretrained('small')
model.lm = model.lm.to(torch.float32)

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
c:\Users\Paarth Tandon\Desktop\repos\personal-music-gen\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data[100]

('../data/chunks_original/animalsasleadersonimpulse_chunk_10_original.wav',
 '../data/chunks_original/animalsasleadersonimpulse_chunk_10_original.txt')

In [7]:
import random

def preprocess_audio(audio_path, model: MusicGen, duration: int = 30):
    wav, sr = torchaudio.load(audio_path)
    wav = torchaudio.functional.resample(wav, sr, model.sample_rate)
    wav = wav.mean(dim=0, keepdim=True)
    if wav.shape[1] < model.sample_rate * duration:
        return None
    end_sample = int(model.sample_rate * duration)
    start_sample = random.randrange(0, max(wav.shape[1] - end_sample, 1))
    wav = wav[:, start_sample : start_sample + end_sample]

    assert wav.shape[0] == 1

    wav = wav.cuda()
    wav = wav.unsqueeze(1)

    with torch.no_grad():
        gen_audio = model.compression_model.encode(wav)

    codes, scale = gen_audio

    assert scale is None

    return codes

In [97]:
orig, _ = torchaudio.load(data[100][0])
orig = orig.mean(dim=0, keepdim=True)
orig = orig.unsqueeze(1).cuda()
orig, _ = model.compression_model.encode(orig)

prep = preprocess_audio(data[100][0], model)

In [105]:
orig.shape, prep.shape

(torch.Size([1, 4, 1500]), torch.Size([1, 4, 1500]))

In [99]:
attributes, _ = model._prepare_tokens_and_attributes(['prompt'], None)
len(attributes), attributes

(1,
 [ConditioningAttributes(text={'description': 'prompt'}, wav={'self_wav': WavCondition(wav=tensor([[[0.]]], device='cuda:0'), length=tensor([0], device='cuda:0'), sample_rate=[32000], path=[None], seek_time=[])}, joint_embed={})])

In [100]:
from audiocraft.modules.conditioners import ClassifierFreeGuidanceDropout

def get_condition_tensor(model, attributes) -> torch.Tensor:
    null_conditions = ClassifierFreeGuidanceDropout(p=1.0)(attributes)
    conditions = attributes + null_conditions
    tokenized = model.lm.condition_provider.tokenize(conditions)
    cfg_conditions = model.lm.condition_provider(tokenized)
    return cfg_conditions

condition_tensors = get_condition_tensor(model, attributes)
condition_tensors

{'description': (tensor([[[ 0.2329,  0.5538,  0.3575,  ..., -0.1976, -0.2641, -0.5404],
           [-0.0173, -0.0040,  0.0010,  ...,  0.0045,  0.0052,  0.0121]],
  
          [[-0.0000,  0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000],
           [-0.0000, -0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0000]]],
         device='cuda:0', grad_fn=<MulBackward0>),
  tensor([[1, 1],
          [0, 0]], device='cuda:0'))}

In [106]:
print(type(condition_tensors))

<class 'dict'>


In [102]:
criterion = torch.nn.CrossEntropyLoss()

with torch.autocast(device_type="cuda", dtype=torch.float16):
    lm_output = model.lm.compute_predictions(
        codes=orig, conditions=[], condition_tensors=condition_tensors
    )
    print(lm_output.logits.device)

    codes = orig[0]
    logits = lm_output.logits[0]
    mask = lm_output.mask[0]

    codes = torch.nn.functional.one_hot(codes, 2048).type(logits.dtype)
    print(codes.shape, logits.shape, mask.shape)

    codes = codes.cuda()
    logits = logits.cuda()
    mask = mask.cuda()
    mask = mask.view(-1)
    masked_logits = logits.view(-1, 2048)[mask]
    masked_codes = codes.view(-1, 2048)[mask]
    loss = criterion(masked_logits, masked_codes)
    print(type(loss))

cuda:0
torch.Size([4, 1500, 2048]) torch.Size([4, 1500, 2048]) torch.Size([4, 1500])
<class 'torch.Tensor'>
